In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install transformers
!pip install fairseq
!pip install fastBPE

     |████████████████████████████████| 2.6 MB 7.1 MB/s 
     |████████████████████████████████| 636 kB 58.0 MB/s 
     |████████████████████████████████| 3.3 MB 62.0 MB/s 
     |████████████████████████████████| 895 kB 31.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.7 MB 7.6 MB/s 
     |████████████████████████████████| 144 kB 39.0 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 112 kB 40.0 MB/s 
     |████████████████████████████████| 74 kB 3.9 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141229 sha256=01ea75570427ac0c0c6a7e62e8f17dae07ffe5e89fd7de76b9f2baa64fa35b20
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime


  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483106 sha256=fa8c20ab72f4015ee93bfc8adf53194c35b9a8a4ff713053b2001629e5845fc4
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
Successfully built fastBPE


In [3]:
!git clone https://github.com/vncorenlp/VnCoreNLP
!pip install vncorenlp

Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 215 (delta 0), reused 0 (delta 0), pack-reused 212
Receiving objects: 100% (215/215), 214.22 MiB | 33.41 MiB/s, done.
Resolving deltas: 100% (76/76), done.
     |████████████████████████████████| 2.6 MB 7.0 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=44d0538af5546e80f5568a10232d0663bd133e1282f964967e6fea0894d32204
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp


In [4]:
from transformers import RobertaForSequenceClassification, RobertaConfig
from tensorflow.keras.preprocessing.sequence import pad_sequences
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from vncorenlp import VnCoreNLP
import pandas as pd
import argparse
import torch
import re

In [8]:
path_config  = '/content/drive/MyDrive/BERT/Config/config.json'
path_model = '/content/drive/MyDrive/BERT/Config/pytorch_model.bin'
path_bpe = '/content/drive/MyDrive/BERT/Config/bpe.codes'
path_vocab = '/content/drive/MyDrive/BERT/Config/dict.txt'

def get_model(path_model= None, path_config = None, path_bpe = None, path_vocab = None):
  config = RobertaConfig.from_pretrained(
      path_config, from_tf=False, num_labels = 3, output_hidden_states=False,
  )
  BERT_SA_NEW = RobertaForSequenceClassification.from_pretrained(
      path_model,
      config=config
  )
  BERT_SA_NEW.cuda()
  BERT_SA_NEW.eval()


  try:
    parser = argparse.ArgumentParser()
    parser.add_argument('--bpe-codes', 
        default=path_bpe,
        required=False,
        type=str,
        help='path to fastBPE BPE'
    )
    args, unknown = parser.parse_known_args()
    bpe = fastBPE(args)
  except:
    bpe = None
    print("load bpe fail")

  try:
    vocab = Dictionary()
    vocab.add_from_file(path_vocab)
  except:
    vocab=None
    print('load vocab fail')
  return BERT_SA_NEW, bpe, vocab

model, bpe, vocab = get_model(path_model, path_config, path_bpe, path_vocab)

In [9]:
rdrsegmenter = VnCoreNLP("/content/VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g')

In [10]:
macp = pd.read_excel('/content/Macp.xlsx')
macp = macp.dropna()
tenct = macp['Tên Công ty'].tolist()
for i in range(len(tenct)):
  tenct[i] = str(tenct[i]).lower()
tenct[:5]
tenma = macp['Mã '].tolist()

def del_test(text):
  year = ['năm 2021', 'năm 2020', 'năm 2019', 'năm 2018', 'năm 2017', 'năm 2016', 'năm 2015', 'năm 2014', 'năm 2013', 'năm 2012', 'năm 2011', 'năm 2010', 'Năm 2021', 'Năm 2020', 'Năm 2019', 'Năm 2018', 'Năm 2017', 'Năm 2016', 'Năm 2015', 'Năm 2014', 'Năm 2013', 'Năm 2012', 'Năm 2011', 'Năm 2010', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']
  month = ['tháng 1', 'tháng 2', 'tháng 3', 'tháng 4', 'tháng 5', 'tháng 6', 'tháng 7', 'tháng 8', 'tháng 9', 'tháng 10', 'tháng 11', 'tháng 12', 'Tháng 1', 'Tháng 2', 'Tháng 3', 'Tháng 4', 'Tháng 5', 'Tháng 6', 'Tháng 7', 'Tháng 8', 'Tháng 9', 'Tháng 10', 'Tháng 11', 'tháng 12']
  quy = ['quý 1', 'quý 2', 'quý 3', 'quý 4', 'Quý 1', 'Quý 2', 'Quý 3', 'Quý 4']
  text = text.replace('Covid-19', 'Covid')
  word_segmented_text = rdrsegmenter.ner(text)[0]
  for char, typ in word_segmentedtext:
    if typ == 'B-ORG' or typ == 'I-ORG' or typ == 'B-PER' or typ == 'I-PER':
      char = char.replace('', ' ')
      text = text.replace(char, 'name')
    if typ == "B-LOC" or typ == "I-LOC":
      if char != 'VN':
        char = char.replace('', ' ')
        text = text.replace(char,'loc')
    if typ == 'O':
      if len(re.findall('\d*.?,?\d+%', char)) > 0:
        text = text.replace(char, 'percent')
      if len(re.findall('\s?(?[A-Z]{3,4})?\s?', char)) > 0 and char != 'USD':
          text = text.replace(char, 'name')
      if char in tenma:
        text = text.replace(char, 'name')
      char = char.replace('', ' ')
      char_lower = char.lower()
      if char_lower in tenct:
        text = text.replace(char, 'name')
  text = text.replace('"', '')
  text = text.replace('”', '')
  text = text.replace('“', '')
  text = text.replace('.', '')
  text = text.replace(',', '')
  text = text.replace('(', '')
  text = text.replace(')', '')
  text = text.replace(':', '')
  text = text.replace('-', ' ')
  text = re.sub('\d{0,2}-?\d{0,2}/\d{1,4}', 'date', text)
  for i in quy:
    text = text.replace(i, 'date')
  for i in year:
    text = text.replace(i, 'date')
  for i in month:
    text = text.replace(i, 'date')
  text = re.sub('\d+ năm ', 'date ', text)
  text = re.sub('\d+ tháng ', 'date ', text)
  text = re.sub(' -?\d+\w?', ' number', text)
  text = text.split()
  for i in range(len(text)):
    if text[i].isdigit():
      text[i] = 'number'
  text = ' '.join(text)
  text1 = text.split()
  for i in range(len(text1)+1):
    try:
      if text1[i][0].isupper() and text1[i+1][0].isupper():
        text = text.replace(text1[i], 'name')
        text = text.replace(text1[i+1], 'name')
    except:
      pass
  text = rdrsegmenter.tokenize(text)
  text = ' '.join([' '.join(x) for x in text])
  text = text.lower()
  return text

In [11]:
def predict(model, bpe, sense, vocab):
  subwords = '<s> ' + bpe.encode(sense) + ' </s>'
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
  encoded_sent = pad_sequences([encoded_sent], maxlen=195, dtype="long", value=0, truncating="post", padding="post")
  mask = [int(token_id > 0) for token_id in encoded_sent[0]]


  encoded_sent = torch.tensor(encoded_sent).cuda()
  mask = torch.tensor(mask).cuda()
  encoded_sent = torch.reshape(encoded_sent, (1, 195))
  mask = torch.reshape(mask, (1, 195))

  with torch.no_grad():
    outputs = model(encoded_sent, 
      token_type_ids=None, 
      attention_mask=mask)
    logits = outputs[0]
  return int(torch.argmax(logits))


In [14]:
sent = 'VIC tăng mạnh, giá trị cổ phiếu tỷ phú Phạm Nhật Vượng nắm giữ đạt xấp xỉ 220.000 tỷ đồng'
predict(model, bpe, sent, vocab)

2